In some other languages,
one can not recover from an error,
or it is difficult to recover from an error,
so one tests input before doing something that could provoke the error.
This technique is called **Look Before You Leap** (LBYL)

For example, in a C program, one must avoid dividing by zero.

Below is a C program that divides by numbers read from a file. When it gets the zero, it crashes.

In [1]:
numbers = '''
2
1
0
-1
-2
'''
with open('numbers', 'w') as f:
    f.write(numbers)

In [2]:
c_program_source_code = r'''
#include <stdlib.h>
#include <stdio.h>

int foo(int x)
{
    return 10 / x;
}

int main(int argc, char *argv[])
{
    int x;
    int y;
    
    while (scanf("%d", &x) == 1) {
        y = foo(x);
        printf("foo(%d) --> %d\n", x, y);
    }        
}
'''

with open('foo.c', 'w') as f:
    f.write(c_program_source_code)

!make foo

cc     foo.c   -o foo


In [3]:
!./foo <numbers

foo(2) --> 5
foo(1) --> 10
Floating point exception (core dumped)


Python is similarly vulnerable.

In [4]:
def foo(x):
    return 10 // x

def main(lines):
    for line in lines:
        x = int(line)
        y = foo(x)
        print("foo(%d) --> %d" % (x, y))
        
main(numbers.split())

foo(2) --> 5
foo(1) --> 10


ZeroDivisionError: integer division or modulo by zero

So one checks before dividing as shown below.

In [5]:
c_program_source_code = r'''
#include <stdlib.h>
#include <stdio.h>

int foo(int x)
{
    if (x == 0)
        return 0;
    else
        return 10 / x;
}

int main(int argc, char *argv[])
{
    int x;
    int y;
    
    while (scanf("%d", &x) == 1) {
        y = foo(x);
        printf("foo(%d) --> %d\n", x, y);
    }        
}
'''

with open('foo.c', 'w') as f:
    f.write(c_program_source_code)

!make foo

cc     foo.c   -o foo


In [6]:
!./foo <numbers

foo(2) --> 5
foo(1) --> 10
foo(0) --> 0
foo(-1) --> -10
foo(-2) --> -5


One can use the same LBYL technique in Python.

In [7]:
def foo(x):
    if x == 0:
        return 0
    else :
        return 10 // x

def main(lines):
    for line in lines:
        x = int(line)
        y = foo(x)
        print("foo(%d) --> %d" % (x, y))
        
main(numbers.split())

foo(2) --> 5
foo(1) --> 10
foo(0) --> 0
foo(-1) --> -10
foo(-2) --> -5


Another technique is to just try stuff,
and if it blows up, do something else.

This technique is called **Easier to Ask Forgiveness than Permission** (EAFP).

Python makes it very easy to do something else
when something blows up.

In [8]:
def foo(x):
    try:
        y = 10 // x
    except ZeroDivisionError:
        y = 0
    return y

def main(lines):
    for line in lines:
        x = int(line)
        y = foo(x)
        print("foo(%d) --> %d" % (x, y))
        
main(numbers.split())

foo(2) --> 5
foo(1) --> 10
foo(0) --> 0
foo(-1) --> -10
foo(-2) --> -5


For that simple example,
EAFP does not have much if any benefit over LBYL.
For that simple example,
there is not much benefit in the size or readability of the code.
However, for more complicated problems,
EAFP lets one write much simpler and readable code.

We will use the example of determining is a string is a valid float for Python.

See
[2.4.6. Floating point literals](https://docs.python.org/3/reference/lexical_analysis.html#floating-point-literals)
for what constitutes a valid float.

    floatnumber   ::=  pointfloat | exponentfloat
    pointfloat    ::=  [digitpart] fraction | digitpart "."
    exponentfloat ::=  (digitpart | pointfloat) exponent
    digitpart     ::=  digit (["_"] digit)*
    fraction      ::=  "." digitpart
    exponent      ::=  ("e" | "E") ["+" | "-"] digitpart

Some code for that follows.

In [9]:
floats = '''
2.
1.
0.
-1.
-2.
17.
.
.17
17e-3
19.e-3
hello
'''
with open('floats', 'w') as f:
    f.write(floats)

In [10]:
import re

In [11]:
def is_float(s, debug=False):
    digit = f'([0-9])'
    digitpart = f'({digit}(_?{digit})*)'                           # digit (["_"] digit)*
    fraction = f'([.]{digitpart})'                                 # "." digitpart
    pointfloat = f'(({digitpart}?{fraction}) | ({digitpart}[.]))'  # [digitpart] fraction | digitpart "."
    exponent = f'([eE][-+]?{digitpart})'                           # ("e" | "E") ["+" | "-"] digitpart
    exponentfloat = f'(({digitpart} | {pointfloat}) {exponent})'   # (digitpart | pointfloat) exponent
    floatnumber = f'^-?({pointfloat} | {exponentfloat})$'          # pointfloat | exponentfloat
    
    if debug:
        res = (
            digit,
            digitpart,
            fraction,
            pointfloat,
            exponent,
            exponentfloat,
            floatnumber,
        )
        for s in res:
            print(repr(s))
            # print(str(s))
            
    float_pattern = re.compile(floatnumber, re.VERBOSE)

    return re.match(float_pattern, s)

In [12]:
if True:
    _ = is_float('', debug=True)

    for s in floats.split():
        print(f'{s!r} -> {bool(is_float(s))}')

'([0-9])'
'(([0-9])(_?([0-9]))*)'
'([.](([0-9])(_?([0-9]))*))'
'(((([0-9])(_?([0-9]))*)?([.](([0-9])(_?([0-9]))*))) | ((([0-9])(_?([0-9]))*)[.]))'
'([eE][-+]?(([0-9])(_?([0-9]))*))'
'(((([0-9])(_?([0-9]))*) | (((([0-9])(_?([0-9]))*)?([.](([0-9])(_?([0-9]))*))) | ((([0-9])(_?([0-9]))*)[.]))) ([eE][-+]?(([0-9])(_?([0-9]))*)))'
'^-?((((([0-9])(_?([0-9]))*)?([.](([0-9])(_?([0-9]))*))) | ((([0-9])(_?([0-9]))*)[.])) | (((([0-9])(_?([0-9]))*) | (((([0-9])(_?([0-9]))*)?([.](([0-9])(_?([0-9]))*))) | ((([0-9])(_?([0-9]))*)[.]))) ([eE][-+]?(([0-9])(_?([0-9]))*))))$'
'2.' -> True
'1.' -> True
'0.' -> True
'-1.' -> True
'-2.' -> True
'17.' -> True
'.' -> False
'.17' -> True
'17e-3' -> True
'19.e-3' -> True
'hello' -> False


In [13]:
def safe_float(s, default=0):
    if is_float(s):
        return float(s)
    else:
        return default

def main(lines):
    total = 0
    for line in lines:
        x = safe_float(line)
        total += x
    print(f'total is {total}')
        
main(open('floats').read().split())


total is 17.206


Now we try EAFP technique below.

In [ ]:
def safe_float(s, default=0.):
    try:
        x = float(s)
    except ValueError:
        x = default
    return x

def main(lines):
    total = 0
    for line in lines:
        x = safe_float(line)
        total += x
    print(f'total is {total}')
        
main(open('floats').read().split())

The EAFP code is much much simpler.

The LBYL version was very complicated.
If there was a bug in the LBYL version, how would you find it?
If you fixed it, how much confidence would you have that your fix is correct?
How hard would it be to have test cases that covered all the edge cases?

EAFP makes code easier to read, simple, and more reliable.
This is what makes EAFP one of Python's superpowers.

---

should:

- always specify at least one exception
- put as little as possible in the try clause

---

In [ ]:
def convert(s, types):
    '''Converts s to the first type in types that works.'''
    for type_ in types:
        try:
            x = type_(s)
        except ValueError:
            continue
        else:
            return x
    raise ValueError

In [ ]:
fields = (
    'John Doe',
    '2',
    '17.',
    '.17',
    'seventeen',
    'float',
    '-17',
)

for s in fields:
    x = convert(s, (int, float, str))
    print(f'{s!r} -> {x!r} of type {x.__class__.__name__}')

---

That works, but is bad.

One should not have naked excepts.

One should specify exactly which exceptions one is going to catch.
I often do that by just provoking the error
and seeing what it complains about
instead of RTFM.

Naked excepts are bad.

Often provoke the error with known bad input
just to see what error is provoked,
then paste that error into the except statement.

In [ ]:
int('-1234.')

In [ ]:
def mogrify(types, s):
    '''Converts s to the first type in types that works.'''
    for type_ in types:
        try:
            x = type_(s)
        except ValueError:
            continue
        break
            
    return x

In [ ]:
for s in input_strings:
    x = mogrify(types, s)
    print('%r -> %r (%s)' % (s, x, x.__class__.__name__))

For example,
determining if a string can be converted to an int is complicated,
so just trying to convert it to a int,
and if it blows up,
do something else.

In [ ]:
def is_int(s, debug=False):
    nonzerodigit = r'([1-9])'    # nonzerodigit ::=  "1"..."9"
    digit = r'([0-9])'           # digit        ::=  "0"..."9"
    bindigit = r'([01])'         # bindigit     ::=  "0" | "1"
    octdigit = r'([0-7])'        # octdigit     ::=  "0"..."7"
    hexdigit = r'([0-9a-fA-F])'  # hexdigit     ::=  digit | "a"..."f" | "A"..."F"
    decinteger = f'(({nonzerodigit}(_?{digit})*) | (0(_?0)*))'  # decinteger   ::=  nonzerodigit (["_"] digit)* | "0"+ (["_"] "0")*
    bininteger = f'(0[bB](_?{bindigit})+)'  # bininteger   ::=  "0" ("b" | "B") (["_"] bindigit)+
    octinteger = f'(0[oO](_?{octdigit})+)'  # octinteger   ::=  "0" ("o" | "O") (["_"] octdigit)+
    hexinteger = f'(0[xX](_?{hexdigit})+)'  # hexinteger   ::=  "0" ("x" | "X") (["_"] hexdigit)+
    integer = f'^({decinteger} | {bininteger} | {octinteger} | {hexinteger})$'  # integer ::=  decinteger | bininteger | octinteger | hexinteger

    if debug:
        res = (
            nonzerodigit,
            digit,
            bindigit,
            octdigit,
            hexdigit,
            decinteger,
            bininteger,
            octinteger,
            hexinteger,
            integer,
        )
        for s in res:
            print(repr(s))
            
    int_pattern = re.compile(integer, re.VERBOSE)

    return re.match(int_pattern, s)

For that simple example,
EAFP does not have much if any benefit over LBYL.
For that simple example,
there is not much benefit in the size or readability of the code.
However, for more complicated problems,
EAFP lets one write much simpler and readable code.

We will use the example of determining is a string is a valid integer for Python.

See
[2.4.5. Integer literals](https://docs.python.org/3/reference/lexical_analysis.html#integer-literals)
for what constitutes a valid int.

    integer      ::=  decinteger | bininteger | octinteger | hexinteger
    decinteger   ::=  nonzerodigit (["_"] digit)* | "0"+ (["_"] "0")*
    bininteger   ::=  "0" ("b" | "B") (["_"] bindigit)+
    octinteger   ::=  "0" ("o" | "O") (["_"] octdigit)+
    hexinteger   ::=  "0" ("x" | "X") (["_"] hexdigit)+
    nonzerodigit ::=  "1"..."9"
    digit        ::=  "0"..."9"
    bindigit     ::=  "0" | "1"
    octdigit     ::=  "0"..."7"
    hexdigit     ::=  digit | "a"..."f" | "A"..."F"
    
Some code for that follows.